In [1]:
from torch import nn
from transformers import TFGPT2LMHeadModel,GPT2Tokenizer
import torch
import tensorflow as tf

#tokenizer = GPT2Tokenizer.from_pretrained("../../Data/Tokenizer/tokenizer-bert-1000.json")
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
model = TFGPT2LMHeadModel.from_pretrained("distilgpt2")

2022-05-09 00:52:18.015606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 00:52:18.078089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 00:52:18.078473: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 00:52:18.080080: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
file_path = '../../Data/Scraped/wiki_data_3.txt'
with open(file_path, encoding="utf-8") as f:
    text = f.read()

tokenized_text = tokenizer.encode(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (7813227 > 1024). Running this sequence through the model will result in indexing errors


In [4]:
examples = []
block_size = 100
for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
    examples.append(tokenized_text[i:i + block_size])


In [5]:
inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])

dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))

BATCH_SIZE = 12
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [6]:
model.fit(dataset, epochs=1)

6511/6511 [==============================] - 2637s 404ms/step - loss: 1.0607 - logits_loss: 1.0607 - logits_accuracy: 0.6650 - past_key_values_1_accuracy: 1.1349e-04 - past_key_values_2_accuracy: 5.1357e-05 - past_key_values_3_accuracy: 4.7414e-05 - past_key_values_4_accuracy: 9.2700e-05 - past_key_values_5_accuracy: 9.4295e-05 - past_key_values_6_accuracy: 3.5741e-05


In [7]:
model.save("mymodel_epoch_1")

INFO:tensorflow:Assets written to: mymodel_epoch_1/assets


INFO:tensorflow:Assets written to: mymodel_epoch_1/assets


In [8]:
model.fit(dataset, epochs=1)

 230/6511 [>.............................] - ETA: 42:46 - loss: 0.9259 - logits_loss: 0.9259 - logits_accuracy: 0.7027 - past_key_values_1_accuracy: 1.3663e-04 - past_key_values_2_accuracy: 5.9472e-05 - past_key_values_3_accuracy: 4.4680e-05 - past_key_values_4_accuracy: 8.6767e-05 - past_key_values_5_accuracy: 1.2489e-04 - past_key_values_6_accuracy: 4.9560e-05

KeyboardInterrupt: 

In [ ]:
model.save("mymodel_epoch_2")

In [ ]:
model.fit(dataset, epochs=1)

In [ ]:
model.save("mymodel_epoch_3")

In [ ]:
model.fit(dataset, epochs=1)

In [ ]:
model.save("mymodel_epoch_4")

In [ ]:
model.fit(dataset, epochs=1)

In [ ]:
model.save("mymodel_epoch_5")

In [ ]:
model=tf.keras.models.load_model("mymodel")


In [9]:
def generate_text(model, tokenizer, start_string, num_generate):
    
        input_eval = tf.expand_dims(tokenizer.encode(start_string), 0)
        token_ids = []

        for i in range(num_generate):
            predictions = tf.squeeze(model.predict(input_eval)[0], 0)

            predicted_id = tf.random.categorical(predictions, 1)[-1, 0].numpy().item()
            
            input_eval = tf.expand_dims([predicted_id], 0)

            token_ids.append(predicted_id)
    
        return start_string + tokenizer.decode(token_ids)

In [10]:
text = "Ես գնացի "

In [11]:
input_ids = tokenizer.encode(text,return_tensors = 'tf')

In [12]:
greedy_output = model.generate(input_ids,
                               max_length = 50,
                               num_beams = 5,
                               early_stopping = True,
                               no_repeat_ngram_size = 5,
                                     
                              # num_return_sequences = 5,
                               )

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [13]:
print(tokenizer.decode(greedy_output[0],skip_special_tokens = True))

Ես գնացի համալսարանում հետ 
